In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import mobilenet_rm_filt_pt as mb_pt
import copy
from torchsummary import summary

In [10]:
model = mb_pt.MobileNetv1()
# summary(model, (3, 32, 32))

In [3]:
def channel_fraction_pruning(model, fraction=0.2):
    print("In channel fraction pruning function")
    mask_dict={}
    #print(model.state_dict().items())
    #print()
    for name, param in model.state_dict().items():
        # print("The name is " + name)
        # print("The param value is below")
        # print(param)
        # print()
        if ((name == "conv1.weight") or (("layers" in name) and ("conv2" in name))):
            print(name)
            score_list = torch.sum(torch.abs(param),dim=(1,2,3)).to('cpu')
            print("score list")
            print(score_list)
            removed_idx = []
            threshold = np.percentile(np.abs(score_list), fraction*100)
            print("threshold value is " + str(threshold))
            for i,score in enumerate(score_list):
                if score < threshold:
                    removed_idx.append(i)
                param[removed_idx,:,:,:] = 0
                mask_dict[name]=torch.where(torch.abs(param) > 0,1,0)
    model.mask_dict = mask_dict

In [4]:
def model_size(model, count_zeros=True):
    total_params = 0
    nonzero_params = 0
    for tensor in model.parameters():
        t = np.prod(tensor.shape)
        nz = np.sum(tensor.detach().cpu().numpy() != 0.0)

        total_params += t
        nonzero_params += nz
    if not count_zeros:
        return int(nonzero_params)
    else:
        return int(total_params)

In [11]:
model.load_state_dict(torch.load('mbnv1_pt.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [12]:
model_size(model)

3217226

In [13]:
for prune_thres in [0.3]:
    print(prune_thres)
    print("about to fraction prune")
    new_model = copy.deepcopy(model)
    channel_fraction_pruning(new_model, prune_thres)
    # break
    print("about to apply mask")
    new_model._apply_mask()
    # break
    print("about to remove channel")
    new_model = mb_pt.remove_channel(new_model)
    # summary(new_model, (3, 32, 32))
    break

0.3
about to fraction prune
In channel fraction pruning function
conv1.weight
score list
tensor([4.6996, 4.6470, 4.7955, 4.1902, 4.7855, 5.1085, 4.8225, 5.0183, 4.7123,
        5.0384, 4.3877, 4.6423, 4.8195, 4.3119, 4.4850, 4.9664, 5.1392, 4.8768,
        4.8272, 4.6533, 4.8690, 4.8585, 5.0395, 4.7349, 5.0591, 4.9254, 4.9032,
        4.8871, 4.7736, 4.8523, 4.6170, 5.1377])
threshold value is 4.719100141525269
layers.0.conv2.weight
score list
tensor([5.3270, 5.6366, 5.3618, 5.7180, 5.3968, 5.2933, 5.0783, 5.3830, 3.7633,
        5.1041, 5.3118, 5.3478, 5.0499, 5.6451, 5.7515, 5.2193, 4.5912, 5.0705,
        5.6936, 5.4505, 5.0237, 5.5487, 4.8535, 5.1959, 4.8054, 5.4607, 4.8562,
        5.5933, 5.2328, 5.4191, 5.7362, 5.3385, 5.5208, 5.5767, 5.2489, 5.8623,
        5.1873, 5.0173, 4.9702, 5.2198, 5.5855, 5.8860, 6.5471, 5.7024, 5.3603,
        4.9945, 5.1122, 6.0886, 5.7956, 5.1882, 5.7194, 5.8757, 5.6322, 5.2551,
        5.7032, 5.2625, 4.9985, 5.1042, 5.2318, 5.7506, 5.3741, 4.8861, 

In [14]:
model_size(new_model)
# 1590857

1590857

In [15]:
writer = SummaryWriter('runs/mb_cifar10')
train_dataset = dsets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(),download=True)
test_dataset = dsets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=128,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=128,shuffle=True)

Files already downloaded and verified


In [16]:
def test(model):
    correct = 0
    total = 0
    total_loss = 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            # images = Variable(images.view(-1, 1, 28, 28)).to(torch.device('cuda'))
            images = Variable(images.view(-1, 3, 32, 32)).to(torch.device('cpu'))
            labels = Variable(labels).to(torch.device('cpu'))
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()

    accuracy = correct.float() / total
    writer.add_scalar("test_accuracy", accuracy)
    # print(, Test accuracy={100*accuracy.data.item():.4f}")
    # test_acc.append(100 * accuracy)
    return 100*accuracy.data.item()

In [17]:
val = test(new_model)
print(val)

13.809999823570251

In [18]:
criterion = nn.CrossEntropyLoss()
criterion_sum = nn.CrossEntropyLoss(reduction='sum')

In [24]:
iteration = 0
def train(epoch, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    global iteration
    model.train()
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        # images = Variable(images.view(-1,1,28,28)).to(torch.device('cuda'))
        images = Variable(images.view(-1,3,32,32)).to(torch.device('cpu'))

        labels = Variable(labels).to(torch.device('cpu'))

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        if (i+1) % 100 == 0:
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f' % (epoch+1, 5, i+1, len(train_dataset)//128, loss.data.item()))
        iteration += 1
    accuracy = correct.float() / total
    writer.add_scalar("train_accuracy", accuracy, epoch)
    print('Accuracy of the model on the 60000 train images: % f %%' % (100*accuracy))
    # train_acc.append(100 * accuracy)
    # train_loss.append(loss)
    return loss

In [25]:
for fine_tune_epoch in range(5):
        train(fine_tune_epoch, new_model)

Epoch: [ 1/ 5], Step: [ 100/ 390], Loss: 1.2609
Epoch: [ 1/ 5], Step: [ 200/ 390], Loss: 0.7697
Epoch: [ 1/ 5], Step: [ 300/ 390], Loss: 1.0843
Accuracy of the model on the 60000 train images:  66.149994 %
Epoch: [ 2/ 5], Step: [ 100/ 390], Loss: 0.4959
Epoch: [ 2/ 5], Step: [ 200/ 390], Loss: 0.7758
Epoch: [ 2/ 5], Step: [ 300/ 390], Loss: 0.5624
Accuracy of the model on the 60000 train images:  79.598000 %
Epoch: [ 3/ 5], Step: [ 100/ 390], Loss: 0.2508


In [ ]:
val = test(new_model)
print(val)